In [2]:
import psycopg2
import pandas as pd

def query(
    query_text, 
    host = "localhost",
    port = 5432,
    database = "postgres",
    user = "postgres",
    password = "postgres"):
    
    with psycopg2.connect(
                        host=host,
                        port=port,
                        database=database,
                        user=user,
                        password=password
                    ) as conn:
        cur = conn.cursor()
        cur.execute(query_text)
        results = cur.fetchall()
        column_names = [desc[0] for desc in cur.description]

    return_df = pd.DataFrame.from_records(results)
    return_df.columns = column_names
    return return_df
 

# Staging Schema

In [28]:
query("""
    SELECT 
        table_name,
        column_name, 
        data_type, 
        is_nullable, 
        character_maximum_length, 
        numeric_precision, 
        numeric_scale
    FROM information_schema.columns
    WHERE table_schema = 'public_staging'
    order by table_name 
    """)

,table_name,column_name,data_type,is_nullable,character_maximum_length,numeric_precision,numeric_scale
0,link_product_map,row_num,bigint,YES,None,64.0,0.0
1,link_product_map,dw_insert_timestamp_utc,timestamp with time zone,YES,None,NaN,NaN
2,link_product_map,page_rank,integer,YES,None,32.0,0.0
3,link_product_map,link_product_map_uuid,text,YES,None,NaN,NaN
4,link_product_map,parent_product_id,text,YES,None,NaN,NaN
5,link_product_map,color,text,YES,None,NaN,NaN
6,link_product_map,card_links,text,YES,None,NaN,NaN
7,link_product_map,page_links,text,YES,None,NaN,NaN
8,link_product_map,request_timestamp_utc,timestamp with time zone,YES,None,NaN,NaN
9,parent_product,thickness,text,YES,None,NaN,NaN


# What are the most popular products and variants?

In [35]:
query("""
Select
p.brand,
p.thickness,
v.shape,
v.size_grp,
round(avg(case when v.popular_variant_flag then 1 else 0 end)*100,2) as pct_variants_popular,
count(1) as count_variants
from public_staging.parent_product p
join public_staging.parent_variant_map v
on p.parent_product_id = v.parent_product_id
group by 1,2,3,4
having count(1) > 50
order by 5 desc
"""
     ).head(10)

,brand,thickness,shape,size_grp,pct_variants_popular,count_variants
0,Rugs USA,1/4 inch,Rectangle,9x12,34.10,349
1,Rugs USA,1/4 inch,Square,5x8,33.85,65
2,Colonial Mills,None,Runner,Runner,33.60,125
3,Rugs USA,1/2 inch,Rectangle,Oversize,30.99,71
4,Rugs USA,1/2 inch,Round,4x6,29.63,54
5,Rugs USA,1/2 inch,Rectangle,9x12,29.29,437
6,Weave & Wander,None,Rectangle,8x10,28.57,56
7,Rugs USA,1/4 inch,Rectangle,10x14,28.57,98
8,Surya,None,Runner,Runner,27.47,91
9,Rugs USA,1 1/4 inch,Rectangle,9x12,25.81,62


# What is the impact of page rank on popularity?

In [41]:
query("""
Select 
round(min_page_rank, -1) as page_rank_bucket,
avg(v.pct_variants_popular) as avg_pct_variants_popular
from public_staging.parent_product p
join (Select parent_product_id,
        min(page_rank) as min_page_rank 
        from public_staging.link_product_map group by 1) l
on l.parent_product_id = p.parent_product_id
join (
    Select parent_product_id, 
    avg(case when popular_variant_flag then 1 else 0 end) as pct_variants_popular,
    count(1) as count_variants
    from public_staging.parent_variant_map
    group by 1) v
on p.parent_product_id = v.parent_product_id
group by 1
order by 1 
""")

,page_rank_bucket,avg_pct_variants_popular
0,0,0.18728342818228063255
1,10,0.19593589610313830366
2,20,0.19797004196723742244
3,30,0.18222549126439286622
4,40,0.17370178840767076061
5,50,0.16026091717581079283
6,60,0.15314096270687340850
7,70,0.16666140339708451164
8,80,0.17580362899511835682
9,90,0.15803321355646937042
